In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import pydicom
import numpy as np
import tensorflow as tf
from Autoencoder import Autoencoder

In [ ]:
filename = "/home/efs-copdgene-data/copdgene/COPDgene_batch_01/COPDGene_A67949_COPDGene_A67949/19000101/Series_002-144330-COPDGene_A67949_INSP_SHARP/1.2.840.113619.2.55.3.3852317604.396.1281051537.826.323.dcm"
ds = pydicom.dcmread(filename)

In [ ]:
img = ds.pixel_array/ds.pixel_array.max()
img = np.expand_dims(img.flatten(), 0)

n_samples = 1
training_epochs = 40
batch_size = 1
display_step = 5
img_size = img.shape[1]

autoencoder = Autoencoder(n_layers=[img_size, 200],
                          transfer_function = tf.nn.softplus,
                          optimizer = tf.train.AdamOptimizer(learning_rate = 0.001))

In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    # Loop over all batches
    for i in range(total_batch):
        # Fit training using batch data
        cost = autoencoder.partial_fit(img)
        # Compute average loss
        avg_cost += cost / n_samples * batch_size
        recon_img = autoencoder.reconstruct(img)

    # Display logs and imgs per epoch step
    if epoch % display_step == 0:
        fig = plt.figure()
        fig.text(0.5, 0.18, 
                 "Epoch: " + '%d, ' % (epoch + 1) + "Loss: " + "{:.2f}".format(avg_cost), 
                 ha='center',
                 fontsize= 'x-large')
        plt.subplot(1, 2, 1)
        plt.imshow(ds.pixel_array, cmap=plt.cm.bone)
        plt.axis('off')
        plt.title('Original Image')

        plt.subplot(1, 2, 2)
        plt.imshow(recon_img.reshape(ds.pixel_array.shape), cmap=plt.cm.bone)
        plt.axis('off')
        plt.title('Reconstructed Image')